In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from sklearn.model_selection import train_test_split


In [ ]:
lines = open('low-high_orig.txt')

with open('low-high.txt',"w") as oF:
    for l in lines:
       words = l.strip().split(",")
       oF.write(("\t".join([" ".join(words[0:6])," ".join(words[6:12]) ])))
       oF.write("\n")



In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    # s = re.sub(r"([.!?])", r" \1", s)
    # s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    #print(s)
    s = re.sub(r'\*', '', s)
    s = re.sub(r'\*', '', s)

    #print(s)
    return s.strip()

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

MAX_LENGTH = 10




def filterPairs(pairs):
    return [pair for pair in pairs ]#if filterPair(pair)]

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [ ]:
input_lang, output_lang, pairs = prepareData('low', 'high')

# Assuming you have a list of pairs called 'pairs'
pairs = [[item for item in sublist if item != ''] for sublist in pairs]

train_pairs, test_pairs = train_test_split(pairs, test_size=0.2, random_state=42)
print((pairs))


Reading lines...
Read 1095 sentence pairs
Trimmed to 1095 sentence pairs
Counting words...
Counted words:
low 76
high 162
[['a3 a32 b7 b62 drb8 drb15', 'a03:01 a32:01 b07:02 b15:01 drb08:01 drb15:01'], ['a3 a32 b7 b44 drb7 drb15', 'a03:01 a32:01 b07:02 b44:02 drb07:01 drb15:01'], ['a1 a3 b7 b8 drb15 drb15', 'a01:01 a03:01 b07:02 b08:01 drb15:01 drb15:01'], ['a2 a32 b27 b60 drb13 drb15', 'a02:01 a32:01 b27:05 b40:01 drb13:02 drb15:01'], ['a2 a3 b7 b62 drb17 drb15', 'a02:01 a03:01 b07:02 b15:01 drb03:01 drb15:01'], ['a3 a24 b13 b35 drb7 drb15', 'a03:01 a24:02 b13:02 b35:03 drb07:01 drb15:01'], ['a3 a29 b7 b35 drb17 drb15', 'a03:01 a29:02 b07:02 b35:01 drb03:01 drb15:01'], ['a24 a68 b8 b51 drb15 drb15', 'a24:02 a68:01 b08:01 b51:01 drb15:01 drb15:01'], ['a26 a32 b38 b60 drb4 drb15', 'a26:01 a32:01 b38:01 b40:01 drb04:04 drb15:01'], ['a3 a24 b7 b39 drb15 drb16', 'a03:01 a24:02 b07:02 b39:01 drb15:01 drb16:01'], ['a2 a2 b60 b51 drb4 drb15', 'a02:01 a02:01 b40:01 b51:01 drb04:01 drb15:01'], 

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.lstm(embedded)
        return output, (hidden, cell)


class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.out(output)
        return output, hidden

class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.lstm = nn.LSTM(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden, decoder_cell = encoder_hidden  # Splitting into hidden and cell state

        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, (decoder_hidden, decoder_cell), attn_weights = self.forward_step(
                decoder_input, (decoder_hidden, decoder_cell), encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, (decoder_hidden, decoder_cell), attentions

    def forward_step(self, input, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(input))

        query = hidden[0].permute(1, 0, 2)  # Using only the hidden state for attention
        context, attn_weights = self.attention(query, encoder_outputs)
        input_lstm = torch.cat((embedded, context), dim=2)

        output, (hidden, cell) = self.lstm(input_lstm, (hidden[0], hidden[1]))
        output = self.out(output)

        return output, (hidden, cell), attn_weights


In [ ]:
def indexesFromSentence(lang, sentence):
    print(lang)
    #return [lang.word2index[word] for word in sentence.split(' ')]
    # Use a list comprehension with a default value of -1 for missing words
    word_indices = [lang.word2index.get(word, -1) for word in sentence.split(' ')]
    # Filter out words with -1 indices (optional)
    return [index for index in word_indices if index != -1]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size,pairs):
    # input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
    input_lang, output_lang, pairs = prepareData('low', 'high', False)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('')

In [ ]:
hidden_size = 512
batch_size = 32



#input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

#encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
#decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)
#train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

#New code with train and test split
train_batch_size = 32
test_batch_size = 1  # Set batch size to 1 for testing to evaluate one sentence at a time

input_lang, output_lang, train_dataloader = get_dataloader(train_batch_size, train_pairs)
_, _, test_dataloader = get_dataloader(test_batch_size, test_pairs)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)
train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)


Reading lines...
Read 1095 sentence pairs
Trimmed to 1095 sentence pairs
Counting words...
Counted words:
low 76
high 162
Reading lines...
Read 1095 sentence pairs
Trimmed to 1095 sentence pairs
Counting words...
Counted words:
low 76
high 162
2m 11s (- 32m 51s) (5 6%) 0.8985
4m 20s (- 30m 24s) (10 12%) 0.0861
6m 35s (- 28m 32s) (15 18%) 0.0221
8m 48s (- 26m 25s) (20 25%) 0.0092
11m 2s (- 24m 16s) (25 31%) 0.0065
13m 15s (- 22m 5s) (30 37%) 0.0048
15m 28s (- 19m 53s) (35 43%) 0.0188
17m 43s (- 17m 43s) (40 50%) 0.0268
19m 55s (- 15m 30s) (45 56%) 0.0048
22m 7s (- 13m 16s) (50 62%) 0.0035
24m 18s (- 11m 3s) (55 68%) 0.0030
26m 30s (- 8m 50s) (60 75%) 0.0030
28m 40s (- 6m 37s) (65 81%) 0.0027
30m 51s (- 4m 24s) (70 87%) 0.0027
33m 2s (- 2m 12s) (75 93%) 0.0026
35m 14s (- 0m 0s) (80 100%) 0.0027


In [ ]:
encoder.eval()
decoder.eval()

AttnDecoderRNN(
  (embedding): Embedding(162, 512)
  (attention): BahdanauAttention(
    (Wa): Linear(in_features=512, out_features=512, bias=True)
    (Ua): Linear(in_features=512, out_features=512, bias=True)
    (Va): Linear(in_features=512, out_features=1, bias=True)
  )
  (lstm): LSTM(1024, 512, batch_first=True)
  (out): Linear(in_features=512, out_features=162, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:

#Get BLEU score
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu_score(encoder, decoder, test_data, input_lang, output_lang):
    references = []
    hypotheses = []

    for pair in test_data:
        input_sentence = pair[0]
        target_sentence = pair[1]

        output_words, _ = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
        references.append([target_sentence.split()])
        hypotheses.append(output_words)

    bleu_score = corpus_bleu(references, hypotheses)
    return bleu_score


test_bleu_score = calculate_bleu_score(encoder, decoder, test_pairs, input_lang, output_lang)
print(f"Test BLEU Score: {test_bleu_score}")


Test BLEU Score: 0.38603241673110905


In [ ]:
#New code with train and test split
train_batch_size = 32
test_batch_size = 1  # Set batch size to 1 for testing to evaluate one sentence at a time

input_lang, output_lang, train_dataloader = get_dataloader(train_batch_size, train_pairs)
_, _, test_dataloader = get_dataloader(test_batch_size, test_pairs)

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)
train(train_dataloader, encoder1, decoder1, 80, print_every=5, plot_every=5)

Reading lines...
Read 1095 sentence pairs
Trimmed to 1095 sentence pairs
Counting words...
Counted words:
low 76
high 162
Reading lines...
Read 1095 sentence pairs
Trimmed to 1095 sentence pairs
Counting words...
Counted words:
low 76
high 162
0m 55s (- 13m 45s) (5 6%) 0.9896
1m 51s (- 12m 59s) (10 12%) 0.1364
2m 47s (- 12m 6s) (15 18%) 0.0380
3m 43s (- 11m 10s) (20 25%) 0.0172
4m 40s (- 10m 16s) (25 31%) 0.0102
5m 35s (- 9m 19s) (30 37%) 0.0064
6m 30s (- 8m 22s) (35 43%) 0.0051
7m 25s (- 7m 25s) (40 50%) 0.0045
8m 19s (- 6m 28s) (45 56%) 0.0039
9m 13s (- 5m 32s) (50 62%) 0.0085
10m 7s (- 4m 36s) (55 68%) 0.0376
11m 2s (- 3m 40s) (60 75%) 0.0066
11m 56s (- 2m 45s) (65 81%) 0.0037
12m 50s (- 1m 50s) (70 87%) 0.0033
13m 46s (- 0m 55s) (75 93%) 0.0029
14m 40s (- 0m 0s) (80 100%) 0.0027


In [ ]:
encoder.eval()
decoder.eval()

AttnDecoderRNN(
  (embedding): Embedding(162, 512)
  (attention): BahdanauAttention(
    (Wa): Linear(in_features=512, out_features=512, bias=True)
    (Ua): Linear(in_features=512, out_features=512, bias=True)
    (Va): Linear(in_features=512, out_features=1, bias=True)
  )
  (lstm): LSTM(1024, 512, batch_first=True)
  (out): Linear(in_features=512, out_features=162, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:

#Get BLEU score
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu_score(encoder, decoder, test_data, input_lang, output_lang):
    references = []
    hypotheses = []

    for pair in test_data:
        input_sentence = pair[0]
        target_sentence = pair[1]

        output_words, _ = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
        references.append([target_sentence.split()])
        hypotheses.append(output_words)

    bleu_score = corpus_bleu(references, hypotheses)
    return bleu_score


test_bleu_score = calculate_bleu_score(encoder1, decoder1, test_pairs, input_lang, output_lang)
print(f"Test BLEU Score: {test_bleu_score}")


Test BLEU Score: 0.046094383842215744
